In [1]:
import pandas as pd
import numpy as np
import plotly.express as px 
import plotly.graph_objects as go


In [49]:
d_costdata = pd.read_csv('costs_parameters_nbnlscenariokm.csv')
# c10_costdata = pd.read_csv('costs_parameters_10pincrease.csv')
# c5_costdata = pd.read_csv('costs_parameters_5pincrease.csv')
# c15_costdata = pd.read_csv('costs_parameters_15pincrease.csv')
gg100_costdata = pd.read_csv('costs_parameters_100greengas.csv')


scenarios = [d_costdata, gg100_costdata]#c5_costdata, c10_costdata, c15_costdata]

In [18]:
#Overview of the Costs for each Group
number= 1

for scenario in scenarios:
    if scenario.equals(d_costdata):
        costs_dict = { 
                    'Group': ["Building and Installations", "Energy Production", "Storage and Conversion", "Carriers", "Infrastructure", "CCUS", "CO2 Emsissions", "Captured Biogenic CO2"],
                    'Default Cost In Billion Euros': [scenario[scenario['Group']=="costs_building_and_installations"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_production"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_storage_and_conversion"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_carriers"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_infrastructure"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_co2"]['Total costs (eur)'].sum(),
                                             scenario['CO2 emission costs (eur)'].sum(),
                                             scenario['Captured biogenic CO2 costs (eur)'].sum()
                                            ]
                    }
        costs_group = pd.DataFrame(costs_dict)
    else:
        costs_dict = { 
                    'Group': ["Building and Installations", "Energy Production", "Storage and Conversion", "Carriers", "Infrastructure", "CCUS", "CO2 Emsissions", "Captured Biogenic CO2"],
                    f'Scenario {number} Cost In Billion Euros': [scenario[scenario['Group']=="costs_building_and_installations"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_production"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_storage_and_conversion"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_carriers"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_infrastructure"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_co2"]['Total costs (eur)'].sum(),
                                             scenario['CO2 emission costs (eur)'].sum(),
                                             scenario['Captured biogenic CO2 costs (eur)'].sum()
                                            ]
                    }
        costs_group_current = pd.DataFrame(costs_dict)
        costs_group = costs_group.merge(costs_group_current, how='inner', on='Group')
        number += 1

total_values = []

for column in range(len(costs_group.columns)-1):
    total_values.append((costs_group.iloc[:, column+1].sum()/1000000000).round(3))
    costs_group.iloc[:, column+1] = (costs_group.iloc[:, column+1]/1000000000).round(3)

total_values.insert(0, 'Total')
total_df = pd.DataFrame([total_values], columns=costs_group.columns)

costs_group = pd.concat([costs_group, total_df], ignore_index=True)

for column in range(len(costs_group.columns)-2):
    costs_group.insert(3*column+3, f'Change between default and scenario {column+1} (billion euros)', (costs_group.iloc[:, 3*column+2] - costs_group.iloc[:, 1]).round(3))
    costs_group.insert(3*column+4, f'Change between default and scenario {column+1} (in %)', ((costs_group.iloc[:,  3*column+3]/costs_group.iloc[:, 1])*100).round(2))
    costs_group[f'Change between default and scenario {column+1} (in %)'] = (costs_group[f'Change between default and scenario {column+1} (in %)']).astype(str) + '%'

costs_group

,Group,Default Cost In Billion Euros,Scenario 1 Cost In Billion Euros,Change between default and scenario 1 (billion euros),Change between default and scenario 1 (in %)
0,Building and Installations,13.256,13.256,0.000,0.0%
1,Energy Production,13.539,13.725,0.186,1.37%
2,Storage and Conversion,18.448,18.448,0.000,0.0%
3,Carriers,31.015,31.155,0.140,0.45%
4,Infrastructure,20.739,20.739,0.000,0.0%
5,CCUS,0.949,0.949,0.000,0.0%
6,CO2 Emsissions,0.173,0.074,-0.099,-57.23%
7,Captured Biogenic CO2,-1.963,-1.963,0.000,-0.0%
8,Total,96.157,96.384,0.227,0.24%


In [ ]:
#Buildings and Installations Cost Breakdown into Subgroups
cbi_subgroups = d_costdata[d_costdata['Group'] == "costs_building_and_installations"]
cbi_subgroups = cbi_subgroups.groupby("Subgroup")['Total costs (eur)'].sum()

cbi_total = (cbi_subgroups.sum()/1000000000).round(3)
total_df = pd.DataFrame({'Subgroup': ['Total'], 'Cost In Billion Euros': [cbi_total]})

cbi_subgroupsdata = {'Subgroup': cbi_subgroups.index.tolist(),
                     'Cost In Billion Euros': cbi_subgroups.values.tolist()
                    }
cbi_subgroups = pd.DataFrame(cbi_subgroupsdata)

cbi_subgroups['Cost In Billion Euros'] = (cbi_subgroups['Cost In Billion Euros']/1000000000).round(3)
cbi_subgroups = pd.concat([cbi_subgroups, total_df], ignore_index=True)
cbi_subgroups


In [19]:
#Breakdown of the infrastructure costs
number = 1
for scenario in scenarios:
    inf_scenario = scenario[scenario['Group']=="costs_infrastructure"]

    hydrogen_carriers = inf_scenario[inf_scenario['Subgroup']=="hydrogen"]['Total costs (eur)'].sum() + scenario[scenario['Subgroup']=="liquid_hydrogen"]['Total costs (eur)'].sum() + scenario[scenario['Subgroup']=="lohc"]['Total costs (eur)'].sum()
    natural_gas =       inf_scenario[inf_scenario['Subgroup']=="lng"]['Total costs (eur)'].sum() + scenario[scenario['Subgroup']=="network_gas"]['Total costs (eur)'].sum()
    electricity =       inf_scenario[inf_scenario['Subgroup']=="electricity"]['Total costs (eur)'].sum()
    ammonia =           inf_scenario[inf_scenario['Subgroup']=="ammonia"]['Total costs (eur)'].sum()
    oil_p =             inf_scenario[inf_scenario['Subgroup']=="diesel"]['Total costs (eur)'].sum()
    heat =              inf_scenario[inf_scenario['Subgroup']=="heat"]['Total costs (eur)'].sum()

    if scenario.equals(d_costdata):
        inf_subgroupsdata = {'Subgroup': ["Natural Gas", "Hydrogen Carriers", "Electricity", "Heat", "Ammonia", "Oil and Products"],
                        'Default Cost In Billion Euros': [natural_gas, hydrogen_carriers, electricity, heat, ammonia, oil_p]
                        }
        inf_subgroups = pd.DataFrame(inf_subgroupsdata)
        
    else:
        inf_subgroupsdata = {'Subgroup': ["Natural Gas", "Hydrogen Carriers", "Electricity", "Heat", "Ammonia", "Oil and Products"],
                        f'Scenario {number} Cost In Billion Euros': [natural_gas, hydrogen_carriers, electricity, heat, ammonia, oil_p]
                        }
        inf_subgroups_current = pd.DataFrame(inf_subgroupsdata)
        inf_subgroups = inf_subgroups.merge(inf_subgroups_current, how='inner', on='Subgroup')
        number +=1

total_values = ['Total']

for column in range(len(inf_subgroups.columns)-1):
    total_values.append((inf_subgroups.iloc[:, column+1].sum()/1000000000).round(3))
    inf_subgroups.iloc[:, column+1] = (inf_subgroups.iloc[:, column+1]/1000000000).round(3)

total_df = pd.DataFrame([total_values], columns=inf_subgroups.columns)

inf_subgroups = pd.concat([inf_subgroups, total_df], ignore_index=True)

for column in range(len(inf_subgroups.columns)-2):
    inf_subgroups.insert(3*column+3, f'Change between default and scenario {column+1} (billion euros)', (inf_subgroups.iloc[:, 3*column+2] - inf_subgroups.iloc[:, 1]).round(3))
    inf_subgroups.insert(3*column+4, f'Change between default and scenario {column+1} (in %)', ((inf_subgroups.iloc[:,  3*column+3]/inf_subgroups.iloc[:, 1])*100).round(2))
    inf_subgroups[f'Change between default and scenario {column+1} (in %)'] = (inf_subgroups[f'Change between default and scenario {column+1} (in %)']).astype(str) + '%'

inf_subgroups

,Subgroup,Default Cost In Billion Euros,Scenario 1 Cost In Billion Euros,Change between default and scenario 1 (billion euros),Change between default and scenario 1 (in %)
0,Natural Gas,1.494,1.494,0.0,0.0%
1,Hydrogen Carriers,10.077,10.077,0.0,0.0%
2,Electricity,14.867,14.867,0.0,0.0%
3,Heat,1.506,1.506,0.0,0.0%
4,Ammonia,0.884,0.884,0.0,0.0%
5,Oil and Products,0.000,0.000,0.0,nan%
6,Total,28.828,28.828,0.0,0.0%


Subcategories hydrogen, liquid_hydrogen, and lohc are combined into one in gquerry under hydrogen_carriers
Subcategories lng and network gas are combined into one category in gquerry under natural_gas
Heat is made up of HT, MT and LT heat
Ammonia is made up of capital and operating expenses
Subcategory diesel appears as oil_and_products in gquery and is breaken down into capital and operating expenses


In [21]:
#Breakdown of energy production costs
number = 1
for scenario in scenarios:
    production_scenario = scenario[scenario['Group']=="costs_production"]

    biomass =                       production_scenario[production_scenario['Subgroup']=="biomass"]['Total costs (eur)'].sum()
    chp_plants =                    production_scenario[production_scenario['Subgroup']=="chp_plants"]['Total costs (eur)'].sum()
    heat_plants =                   production_scenario[production_scenario['Subgroup']=="heat_plants"]['Total costs (eur)'].sum()
    power_plants =                  production_scenario[production_scenario['Subgroup']=="power_plants"]['Total costs (eur)'].sum()
    dedicated_hydrogen_production = production_scenario[production_scenario['Subgroup']=="dedicated_hydrogen_production"]['Total costs (eur)'].sum()
    other_production_cost =         production_scenario[production_scenario['Subgroup']=="other"]['Total costs (eur)'].sum()

    if scenario.equals(d_costdata):
        production_subgroupsdata = {'Subgroup': ["Power Plants", "CHP Plants", "Heat Plants", "Hydrogen Production", "Biomass", "Other"],
                        'Default Cost In Billion Euros': [power_plants, chp_plants, heat_plants, dedicated_hydrogen_production, biomass, other_production_cost]
                        }
        production_subgroups = pd.DataFrame(production_subgroupsdata)
        
    else:
        production_subgroupsdata = {'Subgroup': ["Power Plants", "CHP Plants", "Heat Plants", "Hydrogen Production", "Biomass", "Other"],
                        f'Scenario {number} Cost In Billion Euros': [power_plants, chp_plants, heat_plants, dedicated_hydrogen_production, biomass, other_production_cost]
                        }
        production_subgroups_current = pd.DataFrame(production_subgroupsdata)
        production_subgroups = production_subgroups.merge(production_subgroups_current, how='inner', on='Subgroup')
        number +=1

total_values = ['Total']

for column in range(len(production_subgroups.columns)-1):
    total_values.append((production_subgroups.iloc[:, column+1].sum()/1000000000).round(3))
    production_subgroups.iloc[:, column+1] = (production_subgroups.iloc[:, column+1]/1000000000).round(3)

total_df = pd.DataFrame([total_values], columns=production_subgroups.columns)

production_subgroups = pd.concat([production_subgroups, total_df], ignore_index=True)

for column in range(len(production_subgroups.columns)-2):
    production_subgroups.insert(3*column+3, f'Change between default and scenario {column+1} (billion euros)', (production_subgroups.iloc[:, 3*column+2] - production_subgroups.iloc[:, 1]).round(3))
    production_subgroups.insert(3*column+4, f'Change between default and scenario {column+1} (in %)', ((production_subgroups.iloc[:,  3*column+3]/production_subgroups.iloc[:, 1])*100).round(2))
    production_subgroups[f'Change between default and scenario {column+1} (in %)'] = (production_subgroups[f'Change between default and scenario {column+1} (in %)']).astype(str) + '%'

production_subgroups

,Subgroup,Default Cost In Billion Euros,Scenario 1 Cost In Billion Euros,Change between default and scenario 1 (billion euros),Change between default and scenario 1 (in %)
0,Power Plants,7.573,7.573,0.000,0.0%
1,CHP Plants,0.196,0.196,0.000,0.0%
2,Heat Plants,1.614,1.614,0.000,0.0%
3,Hydrogen Production,2.461,2.461,0.000,0.0%
4,Biomass,1.696,1.882,0.186,10.97%
5,Other,0.000,0.000,0.000,nan%
6,Total,13.539,13.725,0.186,1.37%


In [ ]:
#Breakdown of the carriers costs
number = 1
for scenario in scenarios:
    carriers_scenario = scenario[scenario['Group']=="costs_carriers"]

    if scenario.equals(d_costdata):
        costs_dict = {
            f'Carrier Costs' : carriers_scenario['Subgroup'].tolist(),
            'Default Cost In Billion Euros': carriers_scenario['Total costs (eur)'].tolist()
        }
        carriers_costs = pd.DataFrame(costs_dict)
    else:
        costs_dict =  {
            f'Carrier Costs' : carriers_scenario['Subgroup'].tolist(),
            f'Scenario {number} Cost In Billion Euros': carriers_scenario['Total costs (eur)'].tolist()
        }
        carriers_current_costs = pd.DataFrame(costs_dict)
        carriers_costs = carriers_costs.merge(carriers_current_costs, how='inner', on=f'Carrier Costs')
        number+=1

total_values = ['Total']

for column in range(len(carriers_costs.columns)-1):
    total_values.append((carriers_costs.iloc[:, column+1].sum()/1000000000).round(3))
    carriers_costs.iloc[:, column+1] = (carriers_costs.iloc[:, column+1]/1000000000).round(3)

total_df = pd.DataFrame([total_values], columns=carriers_costs.columns)

carriers_costs = pd.concat([carriers_costs, total_df], ignore_index=True)

for column in range(len(carriers_costs.columns)-2):
    carriers_costs.insert(3*column+3, f'Change between default and scenario {column+1} (billion euros)', (carriers_costs.iloc[:, 3*column+2] - carriers_costs.iloc[:, 1]).round(3))
    carriers_costs.insert(3*column+4, f'Change between default and scenario {column+1} (in %)', ((carriers_costs.iloc[:,  3*column+3]/carriers_costs.iloc[:, 1])*100).round(2))
    carriers_costs[f'Change between default and scenario {column+1} (in %)'] = (carriers_costs[f'Change between default and scenario {column+1} (in %)']).astype(str) + '%'

carriers_costs

,Carrier Costs,Default Cost In Billion Euros,Scenario 1 Cost In Billion Euros,Change between default and scenario 1 (billion euros),Change between default and scenario 1 (in %)
0,biomass,8.496,8.713,0.217,2.55%
1,oil_and_products,11.876,11.876,0.000,0.0%
2,coal_and_products,0.000,0.000,0.000,nan%
3,uranium,0.045,0.045,0.000,0.0%
4,heat,0.000,0.000,0.000,nan%
5,natural_gas,1.446,1.369,-0.077,-5.33%
6,waste,0.000,0.000,0.000,nan%
7,electricity,0.845,0.845,0.000,0.0%
8,hydrogen,-11.882,-11.882,0.000,-0.0%
9,liquid_hydrogen,5.530,5.530,0.000,0.0%


In [52]:
#Breakdown of the CO2 costs
number = 1
for scenario in scenarios:
    if scenario.equals(d_costdata):
        CO2_scenario = scenario[scenario["CO2 emission costs (eur)"] > 0]
        CO2_factors = CO2_scenario['Key'].tolist()
        costs_dict = {
            f'CO2 Costs' : CO2_factors,
            'Default Cost In Billion Euros': CO2_scenario['CO2 emission costs (eur)'].tolist()
        }
        CO2_costs = pd.DataFrame(costs_dict)
    else:
        costs_dict =  {
            f'CO2 Costs' : CO2_factors,
            f'Scenario {number} Cost In Billion Euros': scenario[scenario['Key'].isin(CO2_factors)]['CO2 emission costs (eur)'].tolist()
        }
        CO2_current_costs = pd.DataFrame(costs_dict)
        CO2_costs = CO2_costs.merge(CO2_current_costs, how='inner', on=f'CO2 Costs')
        number+=1

total_values = ['Total']

for column in range(len(CO2_costs.columns)-1):
    total_values.append((CO2_costs.iloc[:, column+1].sum()/1000000000).round(3))
    CO2_costs.iloc[:, column+1] = (CO2_costs.iloc[:, column+1]/1000000000).round(3)

total_df = pd.DataFrame([total_values], columns=CO2_costs.columns)

CO2_costs = pd.concat([CO2_costs, total_df], ignore_index=True)

for column in range(len(CO2_costs.columns)-2):
    CO2_costs.insert(3*column+3, f'Change between default and scenario {column+1} (billion euros)', (CO2_costs.iloc[:, 3*column+2] - CO2_costs.iloc[:, 1]).round(3))
    CO2_costs.insert(3*column+4, f'Change between default and scenario {column+1} (in %)', ((CO2_costs.iloc[:,  3*column+3]/CO2_costs.iloc[:, 1])*100).round(2))
    CO2_costs[f'Change between default and scenario {column+1} (in %)'] = (CO2_costs[f'Change between default and scenario {column+1} (in %)']).astype(str) + '%'

CO2_costs

,CO2 Costs,Default Cost In Billion Euros,Scenario 1 Cost In Billion Euros,Change between default and scenario 1 (billion euros),Change between default and scenario 1 (in %)
0,industry_chemicals_other_burner_crude_oil,0.037,0.037,0.000,0.0%
1,industry_chemicals_other_burner_network_gas,0.037,0.000,-0.037,-100.0%
2,industry_chemicals_refineries_burner_network_gas,0.010,0.000,-0.010,-100.0%
3,industry_other_food_burner_crude_oil,0.003,0.003,0.000,0.0%
4,industry_other_food_burner_network_gas,0.007,0.000,-0.007,-100.0%
5,industry_other_metals_external_coupling_node,0.001,0.000,-0.001,-100.0%
6,industry_other_paper_burner_network_gas,0.002,0.000,-0.002,-100.0%
7,industry_steel_external_coupling_node,0.017,0.000,-0.017,-100.0%
8,industry_heat_backup_burner_network_gas,0.026,0.000,-0.026,-100.0%
9,industry_heat_burner_crude_oil,0.000,0.000,0.000,nan%


In [31]:
def detailed_subgroup(group, subgroup):
    number = 1
    for scenario in scenarios:
        current_scenario = scenario[scenario['Group']==group]
        current_scenario = current_scenario[current_scenario['Subgroup']==subgroup]

        if scenario.equals(d_costdata):
            costs_dict = {
                f'{subgroup} Costs' : current_scenario['Key'].tolist(),
                'Default Cost In Billion Euros': current_scenario['Total costs (eur)'].tolist()
            }
            subgroup_costs = pd.DataFrame(costs_dict)
        else:
            costs_dict =  {
                f'{subgroup} Costs' : current_scenario['Key'].tolist(),
                f'Scenario {number} Cost In Billion Euros': current_scenario['Total costs (eur)'].tolist()
            }
            subgroup_current_costs = pd.DataFrame(costs_dict)
            subgroup_costs = subgroup_costs.merge(subgroup_current_costs, how='inner', on=f'{subgroup} Costs')
            number+=1

    total_values = []

    for column in range(len(subgroup_costs.columns)-1):
        total_values.append((subgroup_costs.iloc[:, column+1].sum()/1000000000).round(3))
        subgroup_costs.iloc[:, column+1] = (subgroup_costs.iloc[:, column+1]/1000000000).round(3)

    total_values.insert(0, 'Total')
    total_df = pd.DataFrame([total_values], columns=subgroup_costs.columns)

    subgroup_costs = pd.concat([subgroup_costs, total_df], ignore_index=True)

    for column in range(len(subgroup_costs.columns)-2):
        subgroup_costs.insert(3*column+3, f'Change between default and scenario {column+1} (billion euros)', (subgroup_costs.iloc[:, 3*column+2] - subgroup_costs.iloc[:, 1]).round(3))
        subgroup_costs.insert(3*column+4, f'Change between default and scenario {column+1} (in %)', ((subgroup_costs.iloc[:,  3*column+3]/subgroup_costs.iloc[:, 1])*100).round(2))
        subgroup_costs[f'Change between default and scenario {column+1} (in %)'] = (subgroup_costs[f'Change between default and scenario {column+1} (in %)']).astype(str) + '%'

    return subgroup_costs

In [37]:
electricity_infcosts = (detailed_subgroup("costs_production", "biomass"))
electricity_infcosts

,biomass Costs,Default Cost In Billion Euros,Scenario 1 Cost In Billion Euros,Change between default and scenario 1 (billion euros),Change between default and scenario 1 (in %)
0,energy_biogas_fermentation_wet_biomass,0.688,0.763,0.075,10.9%
1,energy_greengas_gasification_dry_biomass,0.461,0.512,0.051,11.06%
2,energy_greengas_gasification_wet_biomass,0.363,0.403,0.040,11.02%
3,energy_greengas_upgrade_biogas,0.184,0.204,0.020,10.87%
4,Total,1.696,1.882,0.186,10.97%
